In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

/Users/hemantmehra/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
'''
Source - https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py
'''
import tensorflow as tf

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
'''
    input layer - X - 784 neurons (img shape: 28*28)
    hidden layer 1  - 512 neurons
    hidden layer 2  - 128 neurons
    output layer- Y - 10 neurons (digits: 0-9)
'''
n_input    = 784
n_hidden_1 = 512
n_hidden_2 = 128
n_classes  = 10

# Placeholders
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])

# Layers weights and bias initialization
'''
    Xavier/Glorot Normal initializations
        stddev = sqrt(2/(fan_in + fan_out))
    1. h1 - Input Layer to Hidden Layer 1 (784*512)- stddev = 0.039 
                                                    (sqrt(2/(784 + 512)) = 0.039)
    2. h2 - Hidden Layer 1 to Hidden Layer 2 (512*128) - stddev = 0.055
    3. out - Hidden Layer 2 to Output Layer (128*10) - stddev = 0.120
'''
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev = 0.039, mean = 0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev = 0.055, mean = 0)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev = 0.120, mean = 0))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [5]:
def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    out_layer = tf.nn.sigmoid(out_layer)
    
    return out_layer

logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initializing the variables
init = tf.global_variables_initializer()

In [11]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            avg_cost += c/total_batch
        
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")
    
    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", sess.run(accuracy, feed_dict = {X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001 cost=1.682270460
Epoch: 0002 cost=1.538393069
Epoch: 0003 cost=1.516062083
Epoch: 0004 cost=1.503677753
Epoch: 0005 cost=1.495274047
Epoch: 0006 cost=1.489104148
Epoch: 0007 cost=1.484303362
Epoch: 0008 cost=1.480687835
Epoch: 0009 cost=1.477936466
Epoch: 0010 cost=1.475526191
Epoch: 0011 cost=1.473538216
Epoch: 0012 cost=1.471899954
Epoch: 0013 cost=1.470522485
Epoch: 0014 cost=1.469538789
Epoch: 0015 cost=1.468594079
Optimization Finished!
Accuracy: 0.98
